# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil
import joblib 


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'HyperDrive-Experiment'

experiment=Experiment(ws, experiment_name)

quick-starts-ws-138239
aml-quickstarts-138239
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


# Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

The dataset is loaded in the supporting train.py script using URL


## Hyperdrive Configuration


I have choosen Logistic Regression because it is a binary classification algorithm in which dependent variable is binary i,e 1(True,Sucess),0(False,Failure). Goal is to find the best fitting model for independent and dependent variable in the relationship. 

Here I used RandomParameter Sampling because it randomly select a value for each hyperparameter, which can be a mix of discrete and continuous values.Here in code we need to search for parameter like "_C" and " _max_iter".

PrimaryMetric Goal is used to determine whether a higher value for a metric is better or worse. In this experiment primary metric is **AUC_weighted**.We can maximize or minimize for betterment of model.

Bandit Policy is based on the slack criteria, frequency and delay interval for evaluation.
Slack_factor is the ratio used to calculate the allowed distance from the best performing experiment run.
evaluation_interval, frequency for applying the policy.The main advantage is if we have a certain number of failures, HyperDrive will stop looking for the answers.


In [4]:
# An early termination policy. 
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


# Different parameters that will be used during training
param_sampling = RandomParameterSampling( 
    {
    '--C': choice(0.0001, 0.001, 0.01, 0.1, 1,10,100,1000),
    '--max_iter': choice(100, 200, 300, 400, 500)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"    
    
# Reference: lesson 6.3: copying the training file into the script folder
shutil.copy('./train.py', script_folder)


# Estimator and hyperdrive config
estimator = SKLearn(source_directory = script_folder, 
              compute_target=cpu_cluster, 
              entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, 
                          hyperparameter_sampling=param_sampling,
                          policy=early_termination_policy, 
                          primary_metric_name = "AUC_weighted", 
                          primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs = 15)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


## Submitting the Experiment

In [5]:
hyperdrive_run = experiment.submit(config = hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
#reference: lesson 6.3: running a history widget to show the progress
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_925d200f-1577-4b18-bf7d-3a79793abdaa
Web View: https://ml.azure.com/experiments/HyperDrive-Experiment/runs/HD_925d200f-1577-4b18-bf7d-3a79793abdaa?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-138239/workspaces/quick-starts-ws-138239

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-09T08:50:49.730459][API][INFO]Experiment created<END>\n""<START>[2021-02-09T08:50:50.283460][GENERATOR][INFO]Trying to sample '15' jobs from the hyperparameter space<END>\n""<START>[2021-02-09T08:50:50.700145][GENERATOR][INFO]Successfully sampled '15' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-09T08:50:51.3118737Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_925d200f-1577-4b18-bf7d-3a79793abdaa
Web View: https://ml.azure.com/experiments/HyperDrive-Experiment/runs/HD_925d200f-1577-4b18-bf7d-3a79793abdaa?wsid=/subsc

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
# Getting the best run and displaying its properties
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)
print('Best Run Id: ', best_run.id)
print('AUC_weighted:', best_run_metrics['AUC_weighted'])


['--C', '0.1', '--max_iter', '500']
Best Run Id:  HD_925d200f-1577-4b18-bf7d-3a79793abdaa_1
AUC_weighted: 0.8727777777777778


In [9]:
# Saving the best model
best_run.download_file("/outputs/model.joblib", "LogisticReg_Heart_Failure.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service